tldr - This data has been downloaded from my "saved jobs" on LinkedIn. This notebook helps process and sort the data into our groups for the experiment.

# Summary
I'm running an experiment on how well two different resume's perform. Does a professionally written resume perform better than one I've written myself?			

# Hypothesis
$H_{0} = \text{The resume and cover letter provided by Haley Stock perform the same as my own self-written resume and cover letter.}$

$H_{a} = \text{The professionally-written resume and cover letter have a different interview invitation rate than self-written resume and cover letter.}$

| Name                       | Definition                                                                                                   | Example Format | P1   | P2 (from detectable difference below) |
|----------------------------|-------------------------------------------------------------------------------------------------------------|----------------|------|----------------------------------------|
| Interview Invitation Rate  | This is the percent of applications that receive an interview invite. It shows how interested companies are in my resume. | 0.50%         | 8%   | 13%                                    |
| Application Response Rate  | This is the percent of applications that receive a rejection/interview invite within 48 hrs. It shows how confident companies are in my qualifications. | 1%             |      |                                        |


# Data Collection Process
**LinkedIn:** I searched for "data" and remote in US roles. I then saved the roles that met the criteria below to a csv (via LinkedIn's data request form).

# Inclusion Criteria
Here are the kinds of jobs I'm interested in. I'm going to include these roles in the experiment based on the logic listed below.

| **Category**            | **Details**                                                                                                                                                          |
|:------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Location**            | Remote work only                                                                                                                                                     |
| **Minimum Job Roles**   | Data Scientist, Senior Data/Product Analyst, or Manager/Lead<br>No Applied Scientist, Decision Science, Analytics Engineer, or Researcher roles (significantly different than my experience would allow)<br>No Product Manager roles (not interested in product management)<br>No Director level roles (largely above my level) |
| **Type**                | Full time and contracted                                                                                                                                             |
| **Duplicate Job Roles** | Sometimes companies post multiple versions of a job (by location or reposts after a few weeks have gone by). Will do my best to dedupe these in the final "saved jobs" list. |


# Potential Pitfalls and Adjustments
1) *Cutting sample collection short.* **Discussion:** See power analysis below. Peer review pending.
2) *Confirming random assignment.* **Discussion:** Randomizing at cluster level (company + job title).
3) *Random assignment fails to equally distribute "heavy" segments across A/B.* **Discussion:** Equally distribute the following segments and assess post-assignment:
   - Recency of job posting
   - "Repeat" companies (see "8)" below)
4) *Ensure the Ceteris Parabus assumption* (that is, ensure all else is same between treatment/control besides the treatment). **Discussion:** Use the exact resume and cover letter provided for both.
5) *Cross-contamination for treatment and control groups.* **Discussion:** I don't think we'll have to worry about this, unless a recruiter leaves one company and goes to another.
6) *Multiple comparisons can lead to higher false positive rate.* **Discussion:** Only making one comparison -- professionally written resume vs. self-written resume.
7) *Simpson's paradox due to graduated roll-outs.* **Discussion:** I'm only performing one testing period, so this point is moot. Could simply look at final period if so.
8) *Primacy or novelty effect.* **Discussion:** This could be a problem, if I previously applied for a role with a company before conducting the experiment. I can adjust for this by running the experiment longer, but that's not really helpful given my current situation. Instead, I will include a category segment across treatment/control for repeat and new companies.
    - "Repeat" = companies that I've applied to before. I'll attempt to equally distribute these during assignment.
    - "New" = companies that I have not applied to before. I'll attempt to equally distribute these during assignment.

# Data Processing

In [27]:
!pwd

3626.59s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
/notebooks/resume_experiment


In [50]:
import os
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import StratifiedShuffleSplit

#import the job sampling data
data_folder = "/notebooks/resume_experiment/data"
dataf = "modified_data.csv"
data_file = os.path.join(data_folder, dataf)

print(data_file)

/notebooks/resume_experiment/data/modified_data.csv


In [52]:
raw_df = pd.read_csv(data_file, low_memory=False)
raw_df = raw_df.dropna(axis=0,subset="role_title")
raw_df = raw_df.loc[raw_df.loc[:,'DELETE Row'] != True]
raw_df['sorting_hat_col'] = raw_df['is_repeat_company'].astype('str')+"_"+raw_df['job_posted_pin']+"_"+raw_df['role_cat']

display(raw_df.sample(5))
display(raw_df.loc[raw_df.loc[:,"sorting_hat_col"].isnull()].head(10))

,role_title,company,date_saved,posting_url,days_since_post,is_repeat_company,date_posted,Applied Jobs Check,DELETE Row,Check Unique Job/Co,job_posted_pin,role_cat,sorting_hat_col
100,"Advisor, Data Science and Advance Analytics",Ciena,"11/9/24, 10:51 AM",http://www.linkedin.com/jobs/view/4059612928,6.0,0.0,11/9/2024,NaN,NaN,1,7days,Data Scientist,0.0_7days_Data Scientist
16,"Sr. Manager, Insights & Analytics - Customer S...",Dyson,"11/9/24, 10:39 AM",http://www.linkedin.com/jobs/view/4040507479,2.0,1.0,11/13/2024,NaN,NaN,1,7days,Manager,1.0_7days_Manager
176,Data Scientist,Provation,"11/10/24, 11:25 AM",http://www.linkedin.com/jobs/view/4048274523,7.0,0.0,11/8/2024,NaN,NaN,2,7days,Data Scientist,0.0_7days_Data Scientist
246,"Senior Manager, Data and Analytics - Product",TriNet,"11/5/24, 9:06 AM",http://www.linkedin.com/jobs/view/4049635016,7.0,0.0,11/8/2024,NaN,NaN,1,7days,Manager,0.0_7days_Manager
263,Senior Product Analyst,Greenlight,"11/8/24, 8:39 AM",http://www.linkedin.com/jobs/view/4055136369,21.0,0.0,10/25/2024,NaN,NaN,1,30days,Data Analyst,0.0_30days_Data Analyst


,role_title,company,date_saved,posting_url,days_since_post,is_repeat_company,date_posted,Applied Jobs Check,DELETE Row,Check Unique Job/Co,job_posted_pin,role_cat,sorting_hat_col


In [38]:
#build a list of the companies that are in our dataset. include new vs. repeat as a dimension
companies = raw_df.copy().filter(['company','is_repeat_company','days_since_post'])
companies['counts'] = 1
agg_mets = {'counts':"count", 'days_since_post':"mean"}
companies = companies.groupby(['company','is_repeat_company']).agg(agg_mets).reset_index()
display(companies.head())

,company,is_repeat_company,counts,days_since_post
0,6sense,0.0,1,7.0
1,8x8,0.0,1,14.0
2,Abercrombie & Fitch Co.,0.0,1,14.0
3,Abnormal Security,0.0,1,14.0
4,Age of Learning,0.0,1,6.0


In [39]:
#ensure company lists are mutually exclusive
new_companies = set(new_cluster['company'])
repeat_companies = set(repeat_cluster['company'])

is_mutually_exclusive = new_companies.isdisjoint(repeat_cluster)
assert is_mutually_exclusive, "Companies overlapp between new and repeat."

In [43]:
#Randomly assign clusters to groups
sorting_hat = StratifiedShuffleSplit(n_splits=1, test_size=0.5, random_state=42)

for ctrl, trt in sorting_hat.split(raw_df, raw_df['sorting_hat_col']):
    ctrl_grp = raw_df.iloc[ctrl]
    trt_grp = raw_df.iloc[trt]
    
# Ensure balanced distribution
print("Treatment Group Distribution:")
print(trt_grp['stratify_col'].value_counts())

print("\nControl Group Distribution:")
print(ctrl_grp['stratify_col'].value_counts())

# Treatment and control groups
print("\nTreatment Group Sample:")
print(trt_grp.head())

print("\nControl Group Sample:")
print(ctrl_grp.head())

ValueError: Input contains NaN

In [40]:


for c in clusters:
    generate random number between 1,2
    if random number == 1:
        assign c to control
    elif random number == 2:
        assign c to treatment

save the assignment to a file for control
save the assignment to a file for treatment
tell me how many jobs are in each treatment and control cluster

SyntaxError: invalid syntax (2746964248.py, line 6)